In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Create Google Calendar Events with Gemini and Reasoning Engine in Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/google_calendar_events.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Freasoning-engine%2Fgoogle_calendar_events.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/google_calendar_events.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/reasoning-engine/google_calendar_events.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner) |

## Overview

This is an example application which uses Gemini and Reasoning Engine on Vertex AI to create Google Calendar events from an image of an event poster.

For more information about these features, refer to:

- [Intro to Reasoning Engine Notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/intro_reasoning_engine.ipynb)
- [Reasoning Engine Documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview)

### Objectives

In this tutorial, you will build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python.

You'll build and deploy an agent that uses the Google Calendar API to create events:

- Install the Vertex AI SDK for Python
- Define a model for your agent
- Define Python functions as tools so that our agent can:
   - Create Google Calendar Events
- Use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine
- Test your agent locally before deploying
- Deploy and test your agent on Reasoning Engine in Vertex AI

### Enable APIs and Services

This tutorial uses the following billable components of Google Cloud, which you'll need to enable for this tutorial:

- [Enable Vertex AI API](https://console.cloud.google.com/apis/api/aiplatform.googleapis.com/overview)
- [Enable Calendar API](https://console.cloud.google.com/apis/api/calendar-json.googleapis.com/overview)

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

### Set up Google Calendar API

Follow the [Google Calendar API Python Quickstart](https://developers.google.com/calendar/api/quickstart/python) to set up authentication and create a `credentials.json` file. This will be needed for later steps.

For this tutorial, we are using the [`gcsa`](https://readthedocs.org/projects/google-calendar-simple-api/) Python library to interact with Google Calendar.

## Getting Started


### Install Vertex AI SDK for Python

Install the latest version of the Vertex AI SDK for Python as well as extra dependencies related to Reasoning Engine and LangChain:

In [ ]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-secret-manager" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    gcsa

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
PROJECT_ID = "document-ai-test-337818"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://docai-custom-test"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Example: Build and deploy an agent

### Import libraries

In [19]:
from typing import Optional
from datetime import datetime

from gcsa.google_calendar import GoogleCalendar
from gcsa.event import Event

from vertexai.preview import reasoning_engines
from vertexai.generative_models import GenerativeModel, Part

import tempfile
from google.cloud import secretmanager

from IPython.display import display, Markdown

### Define Python functions (tools)

In this example, you'll define a function called `create_google_calendar_event` that uses the `gsca` library to create Google Calendar events.

In [10]:
def create_google_calendar_event(
    title: str, start: datetime, end: datetime, location: Optional[str], description: Optional[str]
) -> Event:
    """Creates a Google Calendar event."""

    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(
        project=PROJECT_ID, secret="GoogleCalendarCredentials", secret_version="latest"
    )
    response = client.access_secret_version(name=name)
    secret_data = response.payload.data.decode("UTF-8")

    with tempfile.NamedTemporaryFile(mode="w+t", suffix=".json") as temp:
        temp.write(secret_data)
        temp.flush()

        calendar = GoogleCalendar(
            "holtwashere@gmail.com",
            credentials_path=temp.name,
        )
        event = Event(
            title,
            start=start,
            end=end,
            location=location,
            minutes_before_popup_reminder=30,
            description=description.encode().decode("unicode_escape"),
        )
        new_event = calendar.add_event(event)
    return new_event

Test the function with sample inputs to ensure that it's working as expected:

In [ ]:
create_google_calendar_event(
    "Test Event",
    datetime(2024, 6, 24, 11, 21),
    datetime(2024, 6, 24, 12, 21),
    "San Francisco, CA",
    "Awesome event!",
)

### Define model

Here you'll use the Gemini 1.5 Flash model:

In [11]:
model_name = "gemini-1.5-flash-001"

### Define agent

Here, you'll use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine, which brings together the model, tools, and reasoning that you've built up so far:

In [13]:
agent = reasoning_engines.LangchainAgent(
    model=model_name,
    tools=[create_google_calendar_event],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

### Extract Event Information

We will use Gemini to extract event information from an image of a [poster](https://storage.googleapis.com/github-repo/generative-ai/gemini/reasoning-engine/images/EventPoster.jpeg), before we send the information to Reasoning Engine.

In [14]:
model = GenerativeModel(model_name)

In [15]:
image_extraction_prompt = """The following image is an event poster. Extract all of the information from the poster."""

image_part = Part.from_uri(
    "gs://github-repo/generative-ai/gemini/reasoning-engine/images/EventPoster.jpeg",
    mime_type="image/jpeg"
)

response = model.generate_content(contents=[image_extraction_prompt, image_part])

In [20]:
display(Markdown(response.text))

The event is a BBQ celebration. 

**Event Details:**

* **Date:** Sunday, July 7th 2024
* **Time:** 5:00PM - 8:00PM
* **Location:** 10th Floor Club Room
* **Cost:** $20 per guest (2 guests per unit)
* **Caterer:** Pok-E-Jo's
* **Note:** Free for residents

This event is a BBQ celebration, hosted in the 10th Floor Club Room, on July 7th, 2024, from 5:00PM - 8:00PM. Residents have free entry, otherwise it costs $20 per guest, with 2 guests allowed per unit. Pok-E-Jo's will be catering the event.

Now we can test the model and agent behavior to ensure that it's working as expected before we deploy it:

### Test your agent locally

With all of the core components of your agent in place, you can send a prompt to your agent to test that it's working as expected, including the intermediate steps that the agent performed between the input prompt and the generated summary output:

In [21]:
query_input = f"""
Create a Google calendar event based on the following information for an event.:

{response.text}
"""
query_response = agent.query(input=query_input)

print(query_response["output"])

OK. I've created a Google Calendar event for you. 

The event is called "BBQ Celebration" and it's scheduled for Sunday, July 7th, 2024 from 5:00 PM to 8:00 PM in the 10th Floor Club Room. 



### Deploy your agent on Vertex AI

Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

You can re-define the agent to avoid any stateful information in the agent due to our testing in the previous cell:

In [9]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[create_calendar_event],
)

Now you're ready to deploy your agent to Reasoning Engine in Vertex AI by calling `reasoning_engines.ReasoningEngine.create()` along with:

1. The instance of your agent class
2. The Python packages and versions that your agent requires at runtime, similar to how you would define packages and versions in a `requirements.txt` file.

In [ ]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "requests",
        "google-cloud-secret-manager",
        "gcsa",
    ],
)

In [ ]:
remote_agent.query(input=query_input)

### Reusing your deployed agent from other applications or SDKs

You can now import and use the remotely deployed Reasoning Engine in this notebook session or in a different notebook or Python script by uncommenting and adapting the following code:

In [12]:
# from vertexai.preview import reasoning_engines

# PROJECT_ID = "YOUR_PROJECT_ID"
# LOCATION = "YOUR_LOCATION"
# REASONING_ENGINE_ID = "YOUR_REASONING_ENGINE_ID"

# remote_agent = reasoning_engines.ReasoningEngine(f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{REASONING_ENGINE_ID}")
# response = remote_agent.query(input=query)

Or, you can query your agent from other programming languages using any of the [available client libraries in Vertex AI](https://cloud.google.com/vertex-ai/docs/start/client-libraries), including C#, Java, Node.js, Python, Go, or REST API.